In [1]:
from pyspark.sql import SparkSession
import os
import glob
from pyspark.sql.functions import col, year, month, dayofmonth, hour, weekofyear, \
date_format, to_timestamp, dayofweek, concat

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1564429694859_0004,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .appName("Data Frames practice") \
    .getOrCreate()

VBox()

In [3]:
df = spark.read.json('s3a://udacity-dend/song_data/*/*/*')

VBox()

In [4]:
df.count()

VBox()

14896

In [5]:
df.show(n=2)

VBox()

+------------------+---------------+--------------------+----------------+-----------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+-----------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442| Billy Idol|233.22077|        1|SOVIYJY12AF72A4B00|The Dead Next Doo...|1983|
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442| Billy Idol|287.92118|        1|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|1983|
+------------------+---------------+--------------------+----------------+-----------+---------+---------+------------------+--------------------+----+
only showing top 2 rows

In [17]:
df.printSchema()

VBox()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [11]:
df_song_table = df.select(['song_id','title','artist_id','year','duration']).sort(['year','artist_id'])

VBox()

In [12]:
df_song_table.printSchema()

VBox()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [9]:
df_song_table.count()

VBox()

14896

In [15]:
df_song_table.show(n=2, truncate=False)

VBox()

+------------------+---------------------+------------------+----+---------+
|song_id           |title                |artist_id         |year|duration |
+------------------+---------------------+------------------+----+---------+
|SOIBMQP12AB017FF20|Espejismos Y Visiones|AR00FVC1187FB5BE3E|0   |194.66404|
|SOSIUCT12AB0182323|Conversación Casual  |AR00FVC1187FB5BE3E|0   |225.82812|
+------------------+---------------------+------------------+----+---------+
only showing top 2 rows

In [16]:
df_song_table.write.save('s3a://olayemiodefunsho/song_table', format="json", header=True)

VBox()

In [18]:
df_artist_table = df.select(['artist_id','artist_name','artist_location','artist_latitude','artist_longitude'])\
.dropDuplicates()

VBox()

In [19]:
df_artist_table.count()

VBox()

10025

In [20]:
df_artist_table.show(n=2, truncate=False)

VBox()

+------------------+-----------+-----------------------------+---------------+----------------+
|artist_id         |artist_name|artist_location              |artist_latitude|artist_longitude|
+------------------+-----------+-----------------------------+---------------+----------------+
|ARNIDM31187B9A044A|Soulution  |Austin, Honolulu, Los Angeles|34.05349       |-118.24532      |
|ARSBOQJ1187B9B8872|Stan Rogers|Hamilton, Ontario, Canada    |43.26099       |-79.88845       |
+------------------+-----------+-----------------------------+---------------+----------------+
only showing top 2 rows

In [21]:
df_artist_table.write.save('s3a://olayemiodefunsho/artist_table', format="json", header=True)

VBox()

In [23]:
df_log = spark.read.json('s3a://udacity-dend/log_data/*/*/*')

VBox()

In [24]:
df_log.count()

VBox()

8056

In [25]:
df_log.show(n=2, truncate=False)

VBox()

+-----------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+---------------+------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------+------+
|artist     |auth     |firstName|gender|itemInSession|lastName|length   |level|location                          |method|page    |registration     |sessionId|song           |status|ts           |userAgent                                                                                                                                |userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+---------------+------+-------------+---------------------------------------------------------------------------------------------------------------------------

In [26]:
df_log.printSchema()

VBox()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [27]:
df_user_table = df_log.select(['userId','firstName','lastName','gender','level']).where(df_log.userId != '')\
.sort('ts').dropDuplicates(['userId','firstName','lastName'])

VBox()

In [28]:
df_user_table.show(n=2, truncate=False)

VBox()

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|15    |Lily     |Koch    |F     |paid |
|100   |Adler    |Barrera |M     |free |
+------+---------+--------+------+-----+
only showing top 2 rows

In [ ]:
df_user_table.write.save('s3a://olayemiodefunsho/user_table', format="json", header=True)

In [34]:
df_time = df_log.select('ts').dropDuplicates()
df_time = df_time.withColumn('ts', to_timestamp('ts'))
df_time = df_time.select(col("ts").alias("start_time"))
df_time = df_time.withColumn('hour', hour('start_time'))
df_time = df_time.withColumn('day', dayofmonth('start_time'))
df_time = df_time.withColumn('week', weekofyear('start_time'))
df_time = df_time.withColumn('month', month('start_time'))
df_time = df_time.withColumn('year', year('start_time'))
df_time = df_time.withColumn('weekday', dayofweek('start_time'))
df_time = df_time.sort(['year','month'])

VBox()

In [32]:
df_time.count()

VBox()

8023

In [36]:
df_time.show(n=20, truncate=False)

VBox()

+-----------------------+----+---+----+-----+-----+-------+
|start_time             |hour|day|week|month|year |weekday|
+-----------------------+----+---+----+-----+-----+-------+
|50805-09-27 04:43:166.0|4   |27 |39  |9    |50805|3      |
|50805-09-19 01:59:566.0|1   |19 |38  |9    |50805|2      |
|50805-09-16 12:53:166.0|12  |16 |37  |9    |50805|6      |
|50805-09-13 00:59:566.0|0   |13 |37  |9    |50805|3      |
|50805-09-20 17:59:566.0|17  |20 |38  |9    |50805|3      |
|50805-09-16 05:39:566.0|5   |16 |37  |9    |50805|6      |
|50805-09-22 19:09:566.0|19  |22 |38  |9    |50805|5      |
|50805-10-05 01:53:166.0|1   |5  |40  |10   |50805|4      |
|50805-10-14 04:13:166.0|4   |14 |41  |10   |50805|6      |
|50805-10-02 06:56:366.0|6   |2  |39  |10   |50805|1      |
|50805-10-23 11:49:566.0|11  |23 |42  |10   |50805|1      |
|50805-10-21 04:16:366.0|4   |21 |42  |10   |50805|6      |
|50805-10-19 21:43:166.0|21  |19 |42  |10   |50805|4      |
|50805-11-11 19:26:366.0|19  |11 |45  |1

In [37]:
df_time.write.save('s3a://olayemiodefunsho/time_table', format="json", header=True)

VBox()

In [38]:
df_song_play = df_log.join(df, df_log.artist == df.artist_name, 'inner')\
                    .select(concat(df_log.sessionId, df_log.itemInSession).alias('songplay_id'),\
                            to_timestamp(df_log.ts).alias('start_time'),\
                   df_log.userId,df_log.level,df.song_id, df.artist_id,df_log.sessionId,\
                    df.artist_location.alias('location'),df_log.userAgent)

VBox()

In [39]:
df_song_play.count()

VBox()

9957

In [40]:
df_song_play.show(n=20, truncate=False)

VBox()

+-----------+-----------------------+------+-----+------------------+------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+
|songplay_id|start_time             |userId|level|song_id           |artist_id         |sessionId|location                                                                                                                                                                        |userAgent                                                                                                                  |
+-----------+-----------------------+------+-----+------------------+------------------+---------+------------------------------------------------------------------------------------------------------

In [41]:
df_song_play.write.save('s3a://olayemiodefunsho/song_play_table', format="json", header=True)

VBox()